In [46]:
import pandas as pd
from datetime import date
import os

In [30]:
#create an identifier containing the path to the folder
folder = "twitter_data"

#read each file into a dataframe and store them in a list
dfs = []
count = 0
for file_name in os.listdir(folder):
    file_path = os.path.join(folder, file_name)
    df = pd.read_csv(file_path)
    dfs.append(df)
    
#merge the dataframes 
tweets = pd.concat(dfs, axis=0, ignore_index=True)  
tweets.head()


,Tweet_ID,Time_Created,Text,Likes,Retweets,Location
0,1744146496243822931,2024-01-07 23:58:24+00:00,"@SelfMadeMastery Best: $NVDA, $CRWD, $META, $T...",1,0,NaN
1,1744146280576926128,2024-01-07 23:57:32+00:00,Most Notable #Earnings Week of JAN 8th\n\n◦ Mo...,0,0,NaN
2,1744145386850422822,2024-01-07 23:53:59+00:00,🇺🇸 U.S. ECONOMIC DATA 2ND WEEK\n\nTHURS.\n◦ U....,0,0,NaN
3,1744145096592007411,2024-01-07 23:52:50+00:00,$MARA We nailed this play. I kept on hammering...,3,0,"Plan the Trade, Trade the Plan"
4,1743200305033175350,2024-01-05 09:18:34+00:00,$MARA Scaled out of this position yesterday\n\...,1,0,"Plan the Trade, Trade the Plan"


In [31]:
tweets.shape

(287059, 6)

In [33]:
#checking for duplicates
tweets.duplicated(subset=['Tweet_ID']).sum()

16732

In [34]:
#drop duplicates
tweets = tweets.drop_duplicates(subset=['Tweet_ID'], keep='first').reset_index(drop=True)
tweets.shape

(270327, 6)

In [35]:
#making sure duplicates were dropped
tweets.duplicated(subset=['Tweet_ID']).sum()

0

In [36]:
#return the number of missing values in each column of the dataset
tweets.isnull().sum()

Tweet_ID             0
Time_Created         0
Text                 0
Likes                0
Retweets             0
Location        116472
dtype: int64

In [41]:
#drop the location attribute
tweets  = tweets.drop("Location", axis = 1)
tweets.head()

,Tweet_ID,Time_Created,Text,Likes,Retweets
0,1744146496243822931,2024-01-07,"@SelfMadeMastery Best: $NVDA, $CRWD, $META, $T...",1,0
1,1744146280576926128,2024-01-07,Most Notable #Earnings Week of JAN 8th\n\n◦ Mo...,0,0
2,1744145386850422822,2024-01-07,🇺🇸 U.S. ECONOMIC DATA 2ND WEEK\n\nTHURS.\n◦ U....,0,0
3,1744145096592007411,2024-01-07,$MARA We nailed this play. I kept on hammering...,3,0
4,1743200305033175350,2024-01-05,$MARA Scaled out of this position yesterday\n\...,1,0


In [43]:
#Convert the time created attribute to dates 
tweets['Time_Created'] = pd.to_datetime(tweets['Time_Created']).dt.date
tweets.head()

,Tweet_ID,Time_Created,Text,Likes,Retweets
0,1744146496243822931,2024-01-07,"@SelfMadeMastery Best: $NVDA, $CRWD, $META, $T...",1,0
1,1744146280576926128,2024-01-07,Most Notable #Earnings Week of JAN 8th\n\n◦ Mo...,0,0
2,1744145386850422822,2024-01-07,🇺🇸 U.S. ECONOMIC DATA 2ND WEEK\n\nTHURS.\n◦ U....,0,0
3,1744145096592007411,2024-01-07,$MARA We nailed this play. I kept on hammering...,3,0
4,1743200305033175350,2024-01-05,$MARA Scaled out of this position yesterday\n\...,1,0


In [48]:
# Define the start and end dates of the range
start_date = date(2024, 1, 1)
end_date = date(2024, 3, 31)
# Filter the DataFrame based on the date range
filtered_tweets = tweets[(tweets['Time_Created'] >= start_date) & (tweets['Time_Created'] <= end_date)]
filtered_tweets.shape

(269199, 5)

In [49]:
#saving the filtered tweets as a csv file
filtered_tweets.to_csv("twitter_data/NVDA_final-tweets")
print("Filtered tweets added to the folder")

Filtered tweets added to the folder
